## Lab Assignment 3 Scientific Computing

Nick Boon & Marleen Rijksen

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import eigs
from scipy.linalg import eig, eigh
from mpl_toolkits.mplot3d.axes3d import Axes3D

In [ ]:
#From https://rajeshrinet.github.io/blog/2016/gray-scott/
def laplacian(shape):
        """
        Construct a sparse matrix that applies the 5-point discretization
        
        Input:
        shape - tuple of matrix shape (y,x)
        
        Output:
        Sparse matrix that evaluates the 5-point discretization of a shape-sized matrix
        """
        
        M,N = shape
        e=np.ones(N*M)
        e2=([1]*(N-1)+[0])*M
        e3=([0]+[1]*(N-1))*M
        A=spdiags([-4*e,e2,e3,e,e],[0,-1,1,-N,N],N*M,N*M)
        
        return A

In [ ]:
laplacian((5,3)).toarray()

In [ ]:
eigh(laplacian((3,3)).toarray())

### Show eigenmodes in 3D plot

In [ ]:
#from https://stackoverflow.com/a/35126679
def set_aspect_equal_3d(ax):
    """
    Fix equal aspect bug for 3D plots in X and Y.    
    """

    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()

    xmean = np.mean(xlim)
    ymean = np.mean(ylim)

    plot_radius = np.max([abs(lim - mean_)
                       for lims, mean_ in ((xlim, xmean),
                                           (ylim, ymean))
                       for lim in lims])

    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])

shape = (200,200)
a = eigs(laplacian(shape))

#from https://stackoverflow.com/a/8093043
idx = a[0].argsort()[::-1]   

# z = a[1][:,0].reshape(shape)

#need this for interactive 3d plotting!
%matplotlib notebook

#create figure
fig = plt.figure(figsize=(8,8))

for i in idx:
    # set up the axes
    ax = fig.add_subplot(3, 2, i+1, projection='3d')

    # create surface
    x = np.arange(0, shape[1], 1)
    y = np.arange(0, shape[0], 1)
    x, y = np.meshgrid(x, y)
    
    #take only positive values, otherwise eigenmode shape not 
    #clearly visible
    z = np.abs(a[1][:,i].reshape(shape).real)
    
    ax.plot_surface(x,y,z, rstride=3, cstride=3)
    ax.set_title("eigenvalue: %.2f"%(a[0][i].real))
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("amplitude")
    set_aspect_equal_3d(ax)
fig.tight_layout()

### Show eigenmode with heatmap

In [ ]:
#create figure
fig = plt.figure(figsize=(8,8))

for i in idx:
    # set up the axes
    ax = fig.add_subplot(3, 2, i+1)

    #take only positive values, otherwise very disturbing
    #pattern caused by adjecent positive and negative values
    z = np.abs(a[1][:,i].reshape(shape).real)

    ax.imshow(z,origin='lower')
    ax.set_title("eigenvalue: %.2f"%(a[0][i].real))
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_aspect('equal')
fig.tight_layout()